# Défi IA 2023 
## Bedbugs - INSA 5 MA
## Lila, Léa & Flavie

# Code de requêtes


In [14]:
# librairies
# --------------------------------------------
from datetime import datetime
import pandas
import numpy as np
import json
import csv
import time
import requests
import pandas as pd 
import urllib.parse

In [15]:
# connexions à l'api
# --------------------------------------------
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

user_id = 'f80b400f-3d29-43f0-8049-214a756ff0b3' # notre user ID reçu par mail

### Création d'un dataset des requêtes possibles (hors variation des jours (date))
- Stockage dans un fichier csv : allrequest.csv

In [16]:
"""lst1=[0,1]   
lst2=['austrian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']
lst3=['amsterdam', 'copenhagen', 'madrid','paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
print(len(lst2))
lst = [[[j,k,l] for j in lst1] for k in lst2 for l in lst3]

A=np.array(lst)
A=np.reshape(A, (-1,3))
all_requests=pandas.DataFrame(A, columns=["mobile", "language", "city"])
all_requests
all_requests.to_csv('allrequest.csv', index=False)
"""

'lst1=[0,1]   \nlst2=[\'austrian\', \'bulgarian\', \'croatian\', \'cypriot\', \'czech\', \'danish\', \'dutch\', \'estonian\', \'finnish\', \'french\', \'german\', \'greek\', \'hungarian\', \'irish\', \'italian\', \'latvian\', \'lithuanian\', \'luxembourgish\', \'maltese\', \'polish\', \'portuguese\', \'romanian\', \'slovakian\', \'slovene\', \'spanish\',\'swedish\']\nlst3=[\'amsterdam\', \'copenhagen\', \'madrid\',\'paris\', \'rome\', \'sofia\', \'valletta\', \'vienna\', \'vilnius\']\nprint(len(lst2))\nlst = [[[j,k,l] for j in lst1] for k in lst2 for l in lst3]\n\nA=np.array(lst)\nA=np.reshape(A, (-1,3))\nall_requests=pandas.DataFrame(A, columns=["mobile", "language", "city"])\nall_requests\nall_requests.to_csv(\'allrequest.csv\', index=False)\n'

In [17]:
#à faire lors de la première utilisation 
#1) lecture du csv 
df = pandas.read_csv('allrequest.csv', header = 0)
df

,mobile,language,city
0,0,austrian,amsterdam
1,1,austrian,amsterdam
2,0,austrian,copenhagen
3,1,austrian,copenhagen
4,0,austrian,madrid
...,...,...,...
463,1,swedish,valletta
464,0,swedish,vienna
465,1,swedish,vienna
466,0,swedish,vilnius


### Création du fichier de nos requêtes effectuées (pas leur résultats)
- stockage de la liste des requêtes dans un fichier all_our_requests_done.csv

In [18]:
### attention si recompilation : le fichier existant sera écrasé !! 
"""all_our_requests=pandas.DataFrame(columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])
all_our_requests.to_csv('all_our_requests_done.csv', index=False) """

'all_our_requests=pandas.DataFrame(columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])\nall_our_requests.to_csv(\'all_our_requests_done.csv\', index=False) '

### Création du fichier des résultats des requêtes effectuées
- stockage de la liste des requêtes dans un fichier pricing_requests_done.csv

In [19]:
#création du fichier des résultats des requêtes
### attention si recompilation : le fichier existant sera écrasé !! 
"""pricing_requests=pandas.DataFrame(columns=["hotel_id", "price", "stock", "city", "date", "language", "mobile", "avatar_id", "request_number"])"""
"""pricing_requests.to_csv('pricing_requests_done.csv', index=False)"""

"pricing_requests.to_csv('pricing_requests_done.csv', index=False)"

### Bloc de requête à l'api et stockage des données avec les précédentes

In [26]:
#variables 
number_days = 1 # nombre de jour pour lesquels un utilisateur va faire la même requête

#choix du ou des indexes dans la tables des requêtes :
idx_available = [80,15,27,356,90,100]
#nombre de requête
nb_request = 2
chosen_idx = np.random.choice(idx_available, replace = False, size = nb_request)

#chargement de toutes les requêtes possibles et de nos données existantes
our_requests = pandas.read_csv('all_our_requests_done.csv', header = 0)
pricing_requests = pandas.read_csv('pricing_requests_done.csv', header = 0)

#variable qui sert à numéroter nos requêtes (utile si plusieurs requêtes sont faites plusieurs fois)
request_num =our_requests.shape[0]

for i in chosen_idx: #le chosen index correspond à une reqête dans our_requests
    print(i)
    date = datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    days= np.random.choice(range(0, 45), number_days, replace=False)
    j=0
    
     
    #cas 1 avatar fait la même requête sur number_days différents jours
    #création avatar
    time.sleep(1) #pour ne pas avoir 2 avatar de noms identiques
    name = date + '_avatar'     
    #à créer l'avatar 
    a = requests.post(path(f'avatars/{user_id}/{name}'))
    
    for iter_days in (sorted(days, reverse=True)):
        
        #paramètres requête de our_requests
        city=str(df.iloc[i,:]["city"])
        mobile=df.iloc[i,:]["mobile"]
        language=str(df.iloc[i,:]["language"])
        
        #paramètres requetes
        params = {
            "avatar_name": name,
            "language": language,
            "city": city,
            "date": iter_days,
            "mobile": mobile,
        }
        
        print(city)
        print(language)
        
        request_num +=1
        #requête à l'api
        r1 = requests.get(path(f"pricing/{user_id}"), params=params)
        #mise en forme 
        array_to_add = pd.DataFrame(r1.json()['prices']).assign(**r1.json()['request'])
        array_to_add = array_to_add.assign(request_number=request_num)
       
        
        pricing_requests = pd.concat([pricing_requests, array_to_add])
        
        add_to_our_requests = pandas.DataFrame([[r1.json()['request']['avatar_id'], mobile, language, city, iter_days,  i]], 
                                       columns=["avatar_id", "mobile", "language", "city", "date", "id_request"])
        our_requests = pandas.concat([our_requests, add_to_our_requests], ignore_index=True)
        
#enregistrement
pricing_requests.to_csv('pricing_requests_done.csv', index=False)
our_requests.to_csv('all_our_requests_done.csv', index=False)

27
rome
bulgarian
90
amsterdam
danish


In [25]:
#pour quand on s'est raté et que l'enregistrement rate
pricing_requests.to_csv('pricing_requests_done.csv', index=False)
our_requests.to_csv('all_our_requests_done.csv', index=False)